<a href="https://colab.research.google.com/github/isamu-isozaki/toxic-joke-generator/blob/master/GPT_Jokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
model_folder_name = "model"
past_folder_name = "past"
#Makde the folder's of these names in google drive. The current model will be saved to model_folder_name and past models 
#including the current model will be saved to past_folder_name

In [72]:
#cd ..

/content


In [0]:
#!rm -rf toxic-joke-generator
#Used to delete toxic-joke-generator

In [0]:
#Thanks https://www.reddit.com/r/MachineLearning/comments/bgvzdu/d_jokes_generated_via_gpt2/
import os
import json
import random
import re

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
def ListFolder(parent, show=False):
  filelist=[]
  file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % parent}).GetList()
  for f in file_list:
    if show:
      print(f"{f['title']}, {f['id']}")
    if f['mimeType']=='application/vnd.google-apps.folder': # if folder
        if f['title'] == model_folder_name or f['title'] == past_folder_name:          
          filelist.append({"id":f['id'],"title":f['title'], "mimeType": 'application/vnd.google-apps.folder', "list":ListFolder(f['id'])})
        else:
          for f1 in ListFolder(f['id']):
            filelist.append(f1)
    else:
        filelist.append({"id":f['id'],"title":f['title'],"mimeType":"file","title1":f['alternateLink']})
  return filelist
fl = ListFolder('root')

In [74]:
!git clone https://github.com/isamu-isozaki/toxic-joke-generator.git

Cloning into 'toxic-joke-generator'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 127 (delta 15), reused 16 (delta 7), pack-reused 97
Receiving objects: 100% (127/127), 211.18 MiB | 36.49 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Checking out files: 100% (47/47), done.


In [75]:
cd toxic-joke-generator

/content/toxic-joke-generator


In [76]:
!python download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 944kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 57.0Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.20Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:06, 74.1Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 4.93Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 50.8Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 50.5Mit/s]                                                       


In [0]:
if not os.path.exists('checkpoint'):
  os.makedirs("checkpoint")

In [86]:
cd checkpoint

/content/toxic-joke-generator/checkpoint


In [0]:
if not os.path.exists('run1'):
  os.makedirs("run1")

In [0]:
for f in fl:
  if f['title'] == model_folder_name:
    for part in f["list"]:
      fil = drive.CreateFile({'id':part['id']})
      fil.GetContentFile(part['title'][18:])
      os.rename(part['title'][18:], "./run1/"+part['title'][18:])

In [87]:
cd ..

/content/toxic-joke-generator


In [85]:
!pip3 install -r requirements.txt

In [0]:
!python ./train.py --dataset jokes_400_2_3.0_1.txt.npz --batch_size 2 --sample_every 100 --save_every 1000 \
--model_folder_name $model_folder_name --past_model_folder_name $past_folder_name

/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
Get folder list from google drive
2019-06-04 01:09:18.420053: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-04 01:09:18.420266: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x750c680 executing computations on platform Host. Devices:
2019-06-04 01:09:18.420296: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-06-04 01:09:18.585443: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-06-04 01:09:18.585924: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x777b760 executing computation

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

In [0]:
!python3 generate_unconditional_samples.py --top_k 40 --temperature 1.2